In [2]:
import os, sys
sys.path.append(os.path.abspath('../../../src/'))
from utils.daps_explorer import DapsExplorer
from model.CNN import DataLoader, Net, SpectrogramDataset
import matplotlib.pyplot as plt
import wandb

from sklearn.metrics import f1_score
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

wandb.login()

root = DapsExplorer("../../../data/daps/")['clean']['script1']

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mldogs (mldogs-wut). Use `wandb login --relogin` to force relogin


True

# Use wandb to log training progress

In [1]:
INPUT_RESOLUTION = 32
LEARNING_RATE = 0.03
LR_DECAY = 0.95
BATCH_SIZE = 16
DATASET_PATH = os.path.join(os.path.realpath(__file__), '../../../', 'datasets', 'dataset')

In [3]:
import wandb
wandb.login()
wandb.init(
    project="iml_lab1",
    config={
        "learning_rate": LEARNING_RATE,
        "learning_rate_decay": LR_DECAY,
        "batch_size": BATCH_SIZE,
        "input_resolution": INPUT_RESOLUTION,
        # "num_parameters": num_learnable_parameters,
        "architecture": "CNN",
        "dataset": "DAPS"
    }
)


# Simulate training

In [5]:
import random
# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

wandb.finish()

acc,▁▂▆▅▇▇▇█
loss,█▄▂▂▁▁▁▁
acc,0.81641
loss,0.19328


In [1]:
def train():
    transform = transforms.Compose([
        transforms.ToTensor()
    ])

    # Hyperparameters
    batch_size = 32
    learning_rate = 0.001
    num_epochs = 5

    # Prepare Dataset and DataLoader
    train_dataset = SpectrogramDataset(data_dir=os.path.join(DATASET_PATH, "train"),
                                       transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    validation_dataset = SpectrogramDataset(data_dir=os.path.join(DATASET_PATH, "validation"),
                                       transform=transform)
    validation_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    PRINT_STEP = len(train_loader) // 5 - 1

    model = Net()
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_labels = []
        all_preds = []

        for inputs, labels in train_loader:
            # Reshape to match output size [batch_size, 1]
            labels = labels.unsqueeze(1)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track loss and predictions for F1 score
            preds = (outputs > 0.5).float()  # Binary prediction with threshold 0.5
            all_labels.extend(labels.numpy().flatten())
            all_preds.extend(preds.detach().numpy().flatten())
            
            running_loss += loss.item()
            if inputs % PRINT_STEP == PRINT_STEP-1:
                accuracy = correct / PRINT_STEP
                loss = running_loss / PRINT_STEP
                step = epoch * len(train_loader) + inputs
                wandb.log({
                        "train/accuracy": accuracy,
                        "train/loss": loss
                    },
                    step=step
                )
                running_loss = 0.0
                correct = 0

        # Calculate macro-averaged F1 score
        f1 = f1_score(all_labels, all_preds, average='macro')
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}, F1 Score: {f1:.4f}")

        # TODO: model.eval() (validation) 

    torch.save(model, "simple_cnn_full_model.pth")
    print("Training complete.")

In [ ]:
settings = [(m, lr) for m in settings_models for lr in [0.01, 0.003, 0.03]]